https://www.ibm.com/developerworks/jp/linux/library/l-simpy/
https://pypi.python.org/pypi/simpy

In [1]:
import simpy

In [2]:
def clock(env, name, tick):
    while True:
        print(name, env.now)
        yield env.timeout(tick)


env = simpy.Environment()

In [3]:
env.process(clock(env, 'fast', 0.5))

<Process(clock) object at 0x10ebe2860>

In [4]:
env.process(clock(env, 'slow', 1))

<Process(clock) object at 0x10ebe2748>

In [5]:
 env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


https://simpy.readthedocs.io/en/latest/simpy_intro/index.html
https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html

- simpy.Environment: シミュレーション環境
- 単位時間あたりの処理を行う関数（プロセス）をgeneratorとして実装（yieldsを返す)
    - simpy.Environmentを受け取り、Environment.timeoutを返す
- Environment.processでgeneratorを配置
- Environment.run でプロセスが呼ばれる。

In [20]:
def car(env):
    while True:
        print(u"停止: %d" % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('発車:%d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [23]:
# import simpy
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

停止: 0
発車:5
停止: 7
発車:12
停止: 14


https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html

In [31]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())


    def run(self):
        while True:
            print('Start parking and charing at %d' % self.env.now)
            charge_duration = 5
            yield self.env.process(self.charge(charge_duration))
            
            print('Start driving at %d' % self.env.now)
            trip_duration=2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [35]:
# import simpy
env = simpy.Environment()
car = Car(env)
env.run (until=15)

Start parking and charing at 0
Start driving at 5
Start parking and charing at 7
Start driving at 12
Start parking and charing at 14


In [38]:
# ドライバーがとめる動き。
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [37]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())


    def run(self):
        while True:
            print('Start parking and charing at %d' % self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration=2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [43]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15) # 初回は５単位時間かけずにcharge-->driveになっている。

Start parking and charing at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charing at 5
Start driving at 10
Start parking and charing at 12


https://simpy.readthedocs.io/en/latest/simpy_intro/shared_resources.html

In [126]:
def car(env, name, bcs, driving_time, charge_duration):
    yield env.timeout(driving_time)
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req

        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name,env.now))

In [127]:
# import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [128]:
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [129]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html

In [134]:
# import simpy
data = []

def test_process(env, data):
    val = 0
    for i in range(5):
        val += env.now
        data.append(val)
        yield env.timeout(1)

env = simpy.Environment()
p = env.process(test_process(env, data))
env.run(p)
print('Collected', data)

Collected [0, 1, 3, 6, 10]


In [135]:
from functools import partial, wraps
import simpy

def patch_resource(resource, pre=None, post=None):

	def get_wrapper(func):

		@wraps(func)
		def wrapper(*args, **kwargs):
			if pre:
				pre(resource)

			ret = func(*args, **kwargs)

			if post:
				post(resource)

			return ret
		return wrapper
	

	for name in ['put', 'get', 'request', 'release']:
		if hasattr(resource, name):
			setattr(resource, name, get_wrapper(getattr(resource, name)))


def monitor(data, resource):
	item = (
		resource._env.now,
		resource.count,
		len(resource.queue),
	)
	data.append(item)

def test_process(env, res):
	with res.request() as req:
		yield req
		yield env.timeout(1)

env = simpy.Environment()

res = simpy.Resource(env, capacity=1)
data = []

monitor = partial(monitor, data)
patch_resource(res, post=monitor)

p = env.process(test_process(env, res))
env.run(p)

print(data)


[(0, 1, 0), (1, 0, 0)]


- 参考
https://hiroakis.com/blog/2013/07/14/pythonのfunctoolsモジュールデコレータ、部分適用/